In [1]:
#RSA Implementation Generator. Given n bits, get parameters e,d,n for RSA.

def rsa(bits):
    # only prove correctness up to 1024 bits
    proof = (bits <= 1024)
    p = next_prime(ZZ.random_element(2**(bits//2+1)),
            proof=proof)
    q = next_prime(ZZ.random_element(2**(bits//2+1)),
            proof=proof)
    n = p*q
    phi_n = (p-1)*(q-1)
    while True:
        e = ZZ.random_element(1,phi_n)
        if gcd(e,phi_n) == 1: break
    d = lift(Mod(e,phi_n)^(-1))
    return e, d, n 

def rsa_with_small_e(bits): #Adapted RSA implementation with fixed small e to generate example case for parial key exposure attack.
    # only prove correctness up to 1024 bits
    proof = (bits <= 1024)
    p = next_prime(ZZ.random_element(2**(bits//2+1)),
            proof=proof)
    q = next_prime(ZZ.random_element(2**(bits//2+1)),
            proof=proof)
    n = p*q
    phi_n = (p-1)*(q-1)
    while True:
        e = 13
        if gcd(e,phi_n) == 1: break
    d = lift(Mod(e,phi_n)^(-1))
    return e, d, n 


Wiener's attack

In [2]:
def wiener(n,e):
    num = e/n 
    m = 324 # Random message of 324 chosen
    encrypted = power_mod(m,e,n) #Encrypt random message
    for f in num.continued_fraction().convergents(): # Find continued fraction of e/n and convert to convergents
        d = f.denominator() 
        test_m = power_mod(encrypted,d,n) #test if current d is correct
        if test_m == m:
            return d
    
    return 0 #wiener failed most likely due to not meeting condition of d < 1/3* N^1/4


In [3]:
#Example of <N,e> = <90581,17993> Example taken from wikipedia.
wiener(90581,17993)

5

Partial Key Exposure Attack - Coppersmith Theorem

In [4]:
def partial_key_recovery(e,d0,bitlen,N): #Initial part focuses on solving the 3 equations and getting our reduced polynomial.
    
    realp=-1
    realq=-1
    keep=0
    mod = 2^bitlen
    #print(mod)
    R=IntegerModRing(mod)
    
    #Solving ed0 = 1 + k(N-s+1)(mod 2^n/4) for right value of k and s
    for k in range(1,e):
        s = -((e * d0 - 1 - k - k*N) // k) % (2 ^ bitlen)
        #print('s:',s)
        for p in range(e):
            #Plug into p^2-sp+N = 0 mod 2^n/4 and solve for p 
            test = p^2-s*p+N
            shouldBeZero = test % mod
            #print(shouldBeZero)
            if shouldBeZero == 0:
                #print('p0:',p)
                p0 = p
                inverse = inverse_mod(p0,mod)
                q0 = inverse*N % mod
                #print('q0',q0)
                realp, realq = find_pq(mod,p0,q0,N) #p0 and q0 passed into another function to get actual values of p and q
                if realp !=-1:
                    break  
        
        if realp!=-1:
            keep = k
            break;
        else:
            continue
                            
    phi = (realp-1)*(realq-1)
    
    print('Given N = %d, e = %d : ' % (N,e))
    
    print('p:',realp)
    print('q:',realq)
    #print('phi:',phi)
    
    #print(keep)
    d = inverse_mod(e,phi)
    print("Decryption exponent is:",d)
    

def find_pq(mod,p0,q0,N):
    #with p0,q0, we can try to find actual p and q using f(x,y) = (rx+p0)(ry+q0)-N
    for x in range(1,mod+1): #Naive implementation by testing all values of x and y in mod 2^n/4 to find p and q.
        for y in range(1,mod+1):
            testp = (mod*x+int(p0))
            testq = (mod*y+int(q0))
            Ntest = testp*testq
            if Ntest == N:
                return testp,testq
            
    return -1,-1
    
# def find_roots(mod,N,p0):
#     R=IntegerModRing(mod)
#     inverse = inverse_mod(int(p0),mod)
#     #print('inverse:',inverse)
#     q0 = int(IntegerMod(R,inverse*N))
#     print('q0:',q0)
#     for x in range(1,mod+1):
#         for y in range(1,mod+1):
#             testp = (mod*x+int(p0))
#             testq = (mod*y+int(q0))
#             Ntest = testp*testq
#             if Ntest == N:
#                 return testp,testq
#     return -1,-1

In [11]:
rsa_with_small_e(16) #Generate example test case with small e. n=16bits

(13, 8077, 52961)

In [12]:
def test(): #Testing function for partial key exposure attack. Edit e,d,n based on generated example then run function.
    e = 13
    d = 8077
    #p = 263
    #q = 359
    N = 52961
    #phi = (p - 1) * (q - 1)
    #print('phi:',phi)
    n4 = ceil(len(bin(N).replace('0b',''))/4)
    #print('n4:',n4)
    #d = inverse_mod(e,phi)
    #print('d:',d)
    #print(bin(d))
    #print(bin(d)[-n4:])
    d0 = int(bin(d)[-n4:],2)
    #print(d0)
    
    #print(e,d0,n4,N)
    partial_key_recovery(e,d0,n4,N)
    
    
def test2(): #Sometimes fails to find. +1 to n4 can help to solve in some cases.
    e = 13
    p = 449
    q = 1481
    N = 664969
    phi = (p - 1) * (q - 1)
    print('phi:',phi)
    n4 = ceil(len(bin(N).replace('0b',''))/4)
    print('n4:',n4)
    d = inverse_mod(e,phi)
    print('d:',d)
    print(bin(d))
    print(bin(d)[-n4:])
    d0 = int(bin(d)[-n4:],2)
    d02 = int(bin(d)[-(n4+1):],2)
    print(d0)
    print('d2: ',d02)
    
    print(e,d0,n4,N)
    partial_key_recovery(e,d0,n4,N)
    partial_key_recovery(e,d02,n4+1,N)
    
def test3(): #This is just additional testing.
    e = 13
    p = 509
    q = 509
    N = 74593
    phi = (p - 1) * (q - 1)
    print('phi:',phi)
    n4 = ceil(len(bin(N).replace('0b',''))/4)
    print('n4:',n4)
    d = inverse_mod(e,phi)
    print('d:',d)
    print(bin(d))
    print(bin(d)[-n4:])
    d0 = int(bin(d)[-n4:],2)
    d02 = int(bin(d)[-(n4+6):],2)
    print(d0)
    print('d2: ',d02)
    
    print(e,d0,n4,N)
    partial_key_recovery(e,d0,n4,N)
    partial_key_recovery(e,d02,n4+4,N)
    partial_key_recovery(e,259525,18,N)
    
test()

Given N = 52961, e = 13 : 
p: 211
q: 251
Decryption exponent is: 8077
